In [1]:
import random
import string
import copy
import numpy as np
import random as rd

In [2]:
size = 8
w = np.array([10, 18, 12, 14, 13, 11, 8, 6])
gain = np.array([5 , 8 , 7 , 6 , 9 , 5 , 4, 3])
capacity = 35

r = np.max(gain / w) 

In [3]:
# gerar pop. 0 ou 1 aleatorios do tamanho da quantidade de itens

def generate_pop(pop_size):
    pop = []
    for being in range(pop_size):
        pop.append(add_random_asterisk(get_random_string()))
    return pop



In [4]:
def penal(proposed):
    level = r * np.maximum((np.sum(proposed*w) - capacity),0)
    return level 

def fitness(proposed):
    s = []
    for prop in proposed:
        s.append(np.sum(gain*prop) - penal(prop))
    return s

def fitness_nq(proposed):
    v = (np.sum(gain*proposed) - penal(proposed))
    print(v)
    return v

In [5]:
def generate_pop(n = 20):
    pop = []
    for i in range (n):
        ind = np.random.randint(0,2,size)
        pop.append(ind)
    return pop


def mutation(offspring, mutation_rate):
    size_ind = len(offspring[0])
    population_copy = copy.copy(offspring)
    for i in range(len(population_copy)):
        if float(random.randint(0, 10))/10 <= mutation_rate:
            loc = random.randint(0,size_ind-1)
            population_copy[i,loc] = int(not (population_copy[i,loc]))
    return population_copy


def crossover(subset_parents, cross_over_rate):
    new_ = []
    if float(rd.randint(0, 10))/10 <= cross_over_rate:
        size_n = len(subset_parents[0])
        for i in range(0,int(len(subset_parents)/2),2):
            cut_v = random.randint(1, size_n-1)
            son1 = np.concatenate((subset_parents[i][0:cut_v], subset_parents[i+1][cut_v:]))
            son2 = np.concatenate((subset_parents[i+1][0:cut_v], subset_parents[i][cut_v:]))
            new_.append(son1)
            new_.append(son2)
    else:
        new_ = subset_parents
        
    return np.array(new_)

def replacement(offspring_new, pop):
    pop = np.concatenate((pop,offspring_new), axis=0)
    evaluate_v = []
    for indv in pop:
        evaluate_v.append(fitness_nq(indv))
    
    sorted_ = sorted(range(len(evaluate_v)), key=evaluate_v.__getitem__)
    pop_order = pop[sorted_]
    pop_order = pop_order[len(offspring_new):]
    
    return pop_order


def selection_fitness_r(population, indv):
    loss = fitness(population)
    loss_total = np.sum(loss) 
    fitness_norm = loss/loss_total # O fitness dos individuos é normalizado em relação ao fitness da população
    selected = []
    for i in range(indv):
        sel = random.random()
        acc = 0
        prev_acc = 0
        chosen = -1
        for j in range(len(fitness_norm)):
            acc += fitness_norm[j]
            if sel < acc and sel >= prev_acc:
                chosen = j
                break
            prev_acc = acc
        selected.append(population[chosen])
    return selected    

In [6]:
pop = generate_pop(40)
cross_over_rate = 0.7
mutation_rate = 0.7
n_inter_max = 1000
cut = 2
pop

[array([0, 0, 1, 0, 0, 0, 1, 0]),
 array([0, 0, 1, 1, 1, 0, 1, 0]),
 array([1, 0, 1, 1, 1, 1, 1, 1]),
 array([0, 0, 0, 0, 1, 1, 1, 0]),
 array([0, 0, 1, 0, 1, 1, 0, 0]),
 array([0, 0, 0, 0, 0, 1, 0, 1]),
 array([1, 1, 0, 1, 1, 1, 0, 0]),
 array([1, 1, 0, 1, 0, 0, 1, 0]),
 array([1, 1, 1, 0, 1, 1, 1, 0]),
 array([1, 1, 1, 1, 0, 1, 1, 0]),
 array([1, 0, 1, 1, 1, 1, 0, 0]),
 array([0, 1, 0, 0, 1, 0, 0, 1]),
 array([0, 1, 1, 1, 0, 1, 1, 0]),
 array([1, 0, 1, 1, 1, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 1]),
 array([0, 0, 1, 1, 0, 0, 0, 1]),
 array([1, 0, 1, 1, 1, 0, 0, 0]),
 array([0, 1, 0, 0, 1, 1, 1, 1]),
 array([1, 0, 0, 1, 0, 0, 1, 1]),
 array([1, 1, 1, 1, 1, 1, 1, 0]),
 array([1, 0, 1, 0, 1, 0, 1, 1]),
 array([1, 0, 0, 0, 0, 1, 1, 1]),
 array([0, 0, 1, 1, 1, 1, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 1, 0]),
 array([0, 1, 0, 1, 1, 0, 1, 0]),
 array([0, 1, 0, 0, 0, 1, 0, 0]),
 array([0, 1, 0, 1, 1, 0, 0, 0]),
 array([1, 0, 1, 1, 0, 1, 1, 1]),
 array([0, 0, 1, 1, 1, 0, 0, 0]),
 array([1, 1, 

In [7]:
sol = []
n_iter = 0
fitness_avg = []
best_of_generation = []
big_fitness = -1
while n_iter < n_inter_max:
    subset_parents = selection_fitness_r(pop, cut)
    offspring = crossover(subset_parents, cross_over_rate)
    offspring_new = mutation(offspring, mutation_rate)
    pop = replacement(offspring_new, pop)
    fitness_scores = []
    last = fitness_nq(pop[-1])
    
    if last > big_fitness:
        big_fitness = last
        sol = copy.copy(pop[-1])

    best_of_generation.append(big_fitness)
            
    #current_fitness_avg = sum(fitness_scores)/len(fitness_scores)
    #fitness_avg.append(current_fitness_avg)
    print (f"Fitness da geração maior valor {n_iter}: {last}\n")    
    n_iter += 1

print(f"Melhor solução encontrada: {sol}")
print(f"Fitness da melhor solução encontrada: {big_fitness}")

11.0
17.692307692307693
12.0
18.0
20.307692307692307
8.0
11.53846153846154
12.615384615384615
12.384615384615387
8.692307692307693
14.692307692307693
18.615384615384617
10.615384615384617
17.307692307692307
15.0
16.0
17.307692307692307
14.461538461538462
15.923076923076923
8.692307692307693
18.307692307692307
17.0
16.615384615384613
4.0
14.538461538461538
13.0
16.076923076923077
12.0
19.23076923076923
14.076923076923077
15.615384615384615
5.0
13.846153846153847
10.76923076923077
16.923076923076923
11.307692307692307
5.0
13.846153846153847
14.153846153846153
17.0
21.0
15.076923076923077
21.0
Fitness da geração maior valor 0: 21.0

5.0
8.0
8.692307692307693
8.692307692307693
10.615384615384617
10.76923076923077
11.0
11.307692307692307
11.53846153846154
12.0
12.0
12.384615384615387
12.615384615384615
13.0
13.846153846153847
13.846153846153847
14.076923076923077
14.153846153846153
14.461538461538462
14.538461538461538
14.692307692307693
15.0
15.076923076923077
15.615384615384615
15.9230769

19.846153846153847
19.846153846153847
20.0
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
21.0
21.0
21.0
18.23076923076923
19.23076923076923
21.0
Fitness da geração maior valor 39: 21.0

17.307692307692307
17.307692307692307
17.692307692307693
17.692307692307693
17.692307692307693
17.692307692307693
18.0
18.076923076923077
18.23076923076923
18.23076923076923
18.23076923076923
18.23076923076923
18.23076923076923
18.307692307692307
18.307692307692307
18.615384615384617
18.615384615384617
18.76923076923077
19.0
19.0
19.0
19.153846153846153
19.153846153846153
19.153846153846153
19.23076923076923
19.23076923076923
19.23076923076923
19.23076923076923
19.846153846153847
19.846153846153847
20.0
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
21.0
21.0
21.0
17.307692307692307
19.23076923076923
21.0
Fitness da geração maior valor 40: 21.0

17.307692307692307
17.692

19.0
19.153846153846153
19.153846153846153
19.153846153846153
19.153846153846153
19.23076923076923
19.23076923076923
19.23076923076923
19.23076923076923
19.23076923076923
19.23076923076923
19.23076923076923
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
20.0
20.0
20.0
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
21.0
21.0
21.0
21.0
21.0
21.0
21.0
16.923076923076923
19.846153846153847
21.0
Fitness da geração maior valor 67: 21.0

19.0
19.0
19.0
19.0
19.153846153846153
19.153846153846153
19.153846153846153
19.153846153846153
19.23076923076923
19.23076923076923
19.23076923076923
19.23076923076923
19.23076923076923
19.23076923076923
19.23076923076923
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
20.0
20.0
20.0
20.3

19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
20.0
20.0
20.0
20.23076923076923
20.23076923076923
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
19.46153846153846
21.0
Fitness da geração maior valor 107: 21.0

19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
19.846153846153847
20.0
20.0
20.0
20.23076923076923
20.23076923076923
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0


21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
20.307692307692307
17.0
21.0
Fitness da geração maior valor 146: 21.0

20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
18.384615384615387
21.0
Fitness da geração maior valor 147: 21.0

20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
20.307692307692307
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
20.307692307692

...

21.0
21.0
14.0
21.0
Fitness da geração maior valor 816: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
17.307692307692307
21.0
Fitness da geração maior valor 817: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
16.0
16.0
21.0
Fitness da geração maior valor 818: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
19.846153846153847
12.0
21.0
Fitness da geração maior valor 819: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
Fitness da geração maior valor 854: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
12.0
14.0
21.0
Fitness da geração maior valor 855: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
19.846153846153847
19.846153846153847
21.0
Fitness da geração maior valor 856: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
1

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
16.0
21.0
Fitness da geração maior valor 896: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
19.846153846153847
18.384615384615387
21.0
Fitness da geração maior valor 897: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
16.53846153846154
18.384615384615387
21.0
Fitness da geração maior valor 898: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
19.46153846153846
21.0
Fitness da geração maior valor 899: 

21.0
21.0
21.0
19.846153846153847
16.0
21.0
Fitness da geração maior valor 941: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
18.384615384615387
21.0
Fitness da geração maior valor 942: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
Fitness da geração maior valor 943: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
16.0
19.46153846153846
21.0
Fitness da geração maior valor 944: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
2

19.846153846153847
14.0
21.0
Fitness da geração maior valor 975: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
19.46153846153846
21.0
Fitness da geração maior valor 976: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
14.0
21.0
21.0
Fitness da geração maior valor 977: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
18.384615384615387
16.53846153846154
21.0
Fitness da geração maior valor 978: 21.0

21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.0
21.

In [12]:
print('Solution')
print('Número de interações: {}'.format(n_iter))
print('Os objetos utilizados são: {}'.format(sol))
print('Peso na mochila {}'.format(np.sum(sol*w)))
print('Ganho {}'.format(np.sum(sol*gain)))
for e, i in enumerate(sol):
    if i:
        print('{} - Usa'.format(e+1))
    else:
        print('{} - Não'.format(e+1))

Solution
Número de interações: 1000
Os objetos utilizados são: [1 0 1 0 1 0 0 0]
Peso na mochila 35
Ganho 21
1 - Usa
2 - Não
3 - Usa
4 - Não
5 - Usa
6 - Não
7 - Não
8 - Não
